In [ ]:
# Case 1
env = OSWorld()
proc = OSWorldProcessor()

task = next(proc)
env.reset(task)

# Pros
# - one class that handles data ops
# - another for env
# - wanna use benchmark tasks? just use the processor
# - wanna process your custom slice of the benchmarks? just use the processor 
# - wanna use your custom task config? just dont use the processor

# Cons
# - additional class 

In [ ]:
# Case 2
env = OSWorld(OSWorldProcessor("evaluation_examples/"), secrets, use_benchmark=True)

env.reset()

# Pros
# - single class object to handle benchmark tasks or custom tasks

# Cons
# - intermingling between data ops and env ops
# - confuses the init for env; maybe a bit too overloading

In [ ]:
# Case 3

# OSWorld <- OSWorldAdapter <- OSWorldProcessor 
# OSWorldEnv -> OSWorld <- OSWorldProcessor

env = OSWorld()
proc = OSWorldProcessor("evaluation_examples/")

# Pros
# - scalable, modular, robust

# Cons
# - more effort
# - more maintenance
# - more complexity

In [ ]:
example= {
    "id": "94d95f96-9699-4208-98ba-3c3119edf9c2",
    "instruction": "I want to install Spotify on my current system. Could you please help me?",
    "config": [
        {
            "type": "googledrive",
            "parameters": {
                "settings_file": "path/to/googledrive/settings.yml"
            }
        },
        {
            "type": "login",
            "parameters": {
                "settings_file": "evaluation_examples/settings/google/settings.json",
                "platform": "googledrive"
            }
        },
    ],
    "evaluator": {
        "func": "compare_docx_files",
        "result": {
            "type": "googledrive_file",
            "settings_file": "evaluation_examples/settings/googledrive/settings.yml",
            "path": ["environment_policy", "environment_policy_report (draft)"],
            "dest": "environment_policy_report (draft).docx"
        },
        "expected": {
            "type": "cloud_file",
            "path": "https://drive.google.com/uc?id=1A2ti9JncAfIa6ks7FTJWHtYlZo-68FtM&export=download",
            "dest": "environment_policy_report (draft)_gold.docx"
        },
        "options": {
            "content_only": True
        }
    }
}

In [ ]:
env = DesktopEnv()
env.reset()

Googledrive
- settings.yml
  - client_secrets.json
  - credentials.json (write path)

Google
- settings.json

In [2]:
google_credentials_config = {
    "googledrive_settings": {  # settings.yml
        'client_config_backend': 'file',
        'client_config_file': "evaluation_examples/settings/googledrive/client_secrets.json",  # User created.
        'save_credentials': True,
        'save_credentials_backend': 'file',
        'save_credentials_file': "evaluation_examples/settings/googledrive/credentials.json",  # User specified.
        'get_refresh_token': True
    },
    "google_settings": {  # settings.json
        "email": "vincentsimulartest@gmail.com",
        "password": ""
    }
}

# settings.yml (googledrive)
    # client_secrets.json
    # credentials.json path specified by user
# settings.json (google)

In [3]:
google_credentials_config['googledrive_settings']

{'client_config_backend': 'file',
 'client_config_file': 'evaluation_examples/settings/googledrive/client_secrets.json',
 'save_credentials': True,
 'save_credentials_backend': 'file',
 'save_credentials_file': 'evaluation_examples/settings/googledrive/credentials.json',
 'get_refresh_token': True}

In [4]:
google_credentials_config['google_settings']

{'email': 'vincentsimulartest@gmail.com', 'password': ''}